# Property Price Index Analysis

About Property Value and Investment Indexes

Note that there is no standard formula to calculate property price indexes. The formulas differ from Case-Shiller Index, UK Housing Price Index, etc.

**Price to Income Ratio** is a fundamental measure for apartment purchase affordability, where a lower ratio indicates better affordability. It is typically calculated as the ratio of median apartment prices to median familial disposable income, expressed as years of income (although variations are used also elsewhere). Our formula assumes and uses:

- net disposable family income, as defined as 1.5 * the average net salary (50% is the assumed percentage of women in the workforce)
- median apartment size is 90 square meters
- price per square meter (the formula uses) is the average price of square meter in the city center and outside of the city center

**Mortgage as Percentage of Income** is a ratio of the actual monthly cost of the mortgage to take-home family income (lower is better). The average monthly salary is used to estimate family income. It assumes a 100% mortgage is taken on 20 years for the house(or apt) of 90 square meters which price per square meter is the average of prices in the city center and outside of the city center.

**Loan Affordability Index** is an inverse of mortgage as percentage of income. The used formula is : `(100 / mortgage as percentage of income)` (higher is better).

**Price to Rent Ratio** is the average cost of ownership divided by the received rent income (if buying to let) or the estimated rent that would be paid if renting (if buying to reside). Lower values suggest that it is better to buy rather than rent, and higher values suggest that it is better to rent rather than buy. Our formula to estimate rent per square meter assumes 1 bedroom apt has 50 square meters and 3 bedroom apartment has 110 square meters. It doesn't take into account taxes or maintenance fees.

**Gross Rental Yield** is the total yearly gross rent divided by the house price (expressed in percentages). Higher is better.

In [1]:
import pandas as pd
import json

# Set display option to show all rows
pd.set_option("display.max_rows", 100)

In [2]:
df = pd.read_json('data/property_price.json')


In [3]:
df.head()

,Year,City,Price to Income Ratio,Gross Rental Yield City Centre,Gross Rental Yield Outside Centre,Price to Rent Ratio City Centre,Price to Rent Ratio Outside City Centre,Mortgage as a Percentage of Income,Affordability Index
0,2024,"Douala, Cameroon",48.3,10.5,13.0,9.6,7.7,948.9,0.1
1,2024,"Shanghai, China",47.9,1.5,1.5,65.9,66.5,372.2,0.3
2,2024,"Addis Ababa, Ethiopia",45.2,7.0,5.1,14.3,19.7,754.6,0.1
3,2024,"Beirut, Lebanon",41.0,5.0,5.7,20.0,17.6,499.6,0.2
4,2024,"Mumbai, India",38.5,2.1,2.1,48.0,46.9,417.6,0.2


In [4]:
yearly_property_price_df = {}

for year in df['Year'].unique():
    yearly_property_price_df[year] = df[df["Year"] == year].sort_values(by="Price to Income Ratio", ascending=True).reset_index(drop=True)

In [5]:
yearly_property_price_df["2024-mid"].head(20)

,Year,City,Price to Income Ratio,Gross Rental Yield City Centre,Gross Rental Yield Outside Centre,Price to Rent Ratio City Centre,Price to Rent Ratio Outside City Centre,Mortgage as a Percentage of Income,Affordability Index
0,2024-mid,"Detroit, MI, United States",1.2,32.9,21.8,3.0,4.6,10.9,9.2
1,2024-mid,"Rochester, NY, United States",1.8,23.0,20.4,4.3,4.9,16.1,6.2
2,2024-mid,"Indianapolis, IN, United States",1.8,16.1,14.8,6.2,6.7,16.6,6.0
3,2024-mid,"Memphis, TN, United States",1.9,16.4,15.9,6.1,6.3,17.0,5.9
4,2024-mid,"Milwaukee, WI, United States",2.0,20.0,16.8,5.0,6.0,17.8,5.6
5,2024-mid,"Omaha, NE, United States",2.2,17.9,13.6,5.6,7.4,20.1,5.0
6,2024-mid,"Baltimore, MD, United States",2.3,18.6,16.6,5.4,6.0,21.1,4.7
7,2024-mid,"Tulsa, OK, United States",2.3,13.7,11.5,7.3,8.7,20.5,4.9
8,2024-mid,"Atlanta, GA, United States",2.4,12.9,12.6,7.7,7.9,22.0,4.6
9,2024-mid,"Houston, TX, United States",2.5,12.6,13.7,7.9,7.3,22.1,4.5


In [43]:
def plot_graph(df, city):
    fig = make_subplots(rows=4, cols=2, start_cell="top-left", subplot_titles=df.columns[1:])
    fig.update_layout(title=city, height=1500, width= 1500)
    curr_row = 1
    curr_col = 1
    for col in df.columns[1:]:
        fig.add_trace(
            go.Scatter(x=df["Year"], y=df[col], mode="lines+markers", name=col),
            row=curr_row,
            col=curr_col,
        )
        fig.update_xaxes(title_text="Year", row=curr_row, col=curr_col)
        fig.update_yaxes(title_text=col, row=curr_row, col=curr_col)
        curr_col += 1
        if curr_col == 3:
            curr_row += 1
            curr_col = 1
            
    fig.show()
    pio.write_image(fig, f"plots/{city}.png")

In [44]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

cities = ["Detroit, MI, United States", "Rochester, NY, United States"]

In [45]:
df[df["City"] == cities[0]].sort_values("Year", ascending=True)

,Year,City,Price to Income Ratio,Gross Rental Yield City Centre,Gross Rental Yield Outside Centre,Price to Rent Ratio City Centre,Price to Rent Ratio Outside City Centre,Mortgage as a Percentage of Income,Affordability Index
7395,2011,"Detroit, MI, United States",2.6,12.0,14.5,8.4,6.9,23.1,4.3
7690,2012,"Detroit, MI, United States",3.0,11.6,10.6,8.6,9.5,25.2,4.0
7712,2013,"Detroit, MI, United States",1.9,10.4,12.9,9.7,7.8,14.4,6.9
9014,2014,"Detroit, MI, United States",1.2,19.1,17.4,5.2,5.7,9.5,10.5
8917,2014-mid,"Detroit, MI, United States",1.3,19.2,17.7,5.2,5.6,10.1,9.9
9063,2015,"Detroit, MI, United States",1.4,22.6,18.3,4.4,5.5,10.4,9.6
9330,2015-mid,"Detroit, MI, United States",1.9,20.6,13.7,4.9,7.3,13.9,7.2
5923,2016,"Detroit, MI, United States",1.3,20.5,18.3,4.9,5.5,9.5,10.5
5970,2017,"Detroit, MI, United States",1.2,27.4,23.4,3.7,4.3,8.9,11.2
6228,2017-mid,"Detroit, MI, United States",1.0,32.9,26.9,3.0,3.7,7.5,13.4


In [46]:
for city in cities:
    selected_city_df = df[df["City"] == city].sort_values('Year', ascending=True)
    selected_city_df.drop(columns=["City"], inplace=True)
    plot_graph(selected_city_df, city)

Index(['Price to Income Ratio', 'Gross Rental Yield City Centre',
       'Gross Rental Yield Outside Centre', 'Price to Rent Ratio City Centre',
       'Price to Rent Ratio Outside City Centre',
       'Mortgage as a Percentage of Income', 'Affordability Index'],
      dtype='object')


Index(['Price to Income Ratio', 'Gross Rental Yield City Centre',
       'Gross Rental Yield Outside Centre', 'Price to Rent Ratio City Centre',
       'Price to Rent Ratio Outside City Centre',
       'Mortgage as a Percentage of Income', 'Affordability Index'],
      dtype='object')


As an investor:
- Price to income ratio:
    - Higher the ratio:
        - Potential appreciation: High price-to-income ratios often occur in desirable areas with strong demand (e.g., large cities), which could lead to property appreciation over time.
        - Limited rental pool: The higher prices may mean fewer local residents can afford to buy homes, increasing the demand for rentals. However, if rents can't rise with property prices, this could limit your rental income.
        - Risk of market correction: In extreme cases, a high ratio can signal a housing bubble, increasing the risk of price drops in the future.
    - Lower the ratio:
        - Limited appreciation: These markets may not see as much property value appreciation compared to high-ratio markets, making it better for investors focused on income rather than capital gains
        - Better cash flow: Properties with lower price-to-income ratios might be more affordable to purchase and could still offer strong rental income. This leads to better cash flow.
        - Steadier rental demand: In markets with a low ratio, more people may be able to afford to rent or buy, which could keep rental demand steady.
- Price to rent ratio:
    - The higher the ratio: indicates that property prices are rising faster than rents
    - The lower the ratio: Generally more favorable for investors focused on cash flow, as you’re getting more rental income for the price of the property.
- Gross rental yield:
    - Higher the yield: The better the cash flow
    - Lower the yield: The lower the cash flow
    - Industry Benchmark: A gross rental yield of 5-8% is often considered reasonable for residential properties, though this can vary widely by market. For example, urban areas with high appreciation potential may have yields under 5%, while properties in more affordable or emerging markets may have yields above 8%.
- Mortgage as a percentage of income:
    - Low Mortgage-to-Income Ratio:
        - More cash flow, less risk, less leverage
    - High mortgage-to-income ratio:
        - Less cash flow, more risk, more leverage